In [289]:
import numpy as np
import pandas as pd

In [290]:
df = pd.read_csv('mail_data.csv')

In [291]:
df.tail(3)

,Category,Message
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...
5571,ham,Rofl. Its true to its name


In [292]:
import string
from nltk.corpus import stopwords

In [293]:
import nltk

In [294]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lekka\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [295]:
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\lekka\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lekka\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\lekka\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [296]:
from nltk.tokenize import word_tokenize
import nltk.stem
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [297]:

exclude = string.punctuation
stop_words = stopwords.words('english')
word_net_Lemma = WordNetLemmatizer()
def rem_stop_words(text):
    l =[]
    for word in text:
        if word not in stop_words:
            l.append(word)
    text = ' '.join(l)
    return text
    
def preprocess(text):
    text = text.lower()
    text = text.translate(str.maketrans('','',exclude))
    text =  word_tokenize(text)
    text = rem_stop_words(text)
    
    final_sen = []
    for word in text.split():
        final_sen.append(word_net_Lemma.lemmatize(word))
    return ' '.join(final_sen)

In [298]:
df['convt_message'] = df['Message'].apply(preprocess)

In [299]:
df.head(3)

,Category,Message,convt_message
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...


In [300]:
from sklearn.preprocessing import LabelEncoder

In [301]:
le = LabelEncoder()

In [302]:
df['target'] = le.fit_transform(df['Category'])

In [303]:
df['Category'].value_counts()

Category
ham     4825
spam     747
Name: count, dtype: int64

In [304]:
df.head(5)

,Category,Message,convt_message,target
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...,0
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...,1
3,ham,U dun say so early hor... U c already then say...,u dun say early hor u c already say,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah dont think go usf life around though,0


In [305]:
df['convt_message'][0]

'go jurong point crazy available bugis n great world la e buffet cine got amore wat'

In [306]:
df['Message'][0]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [307]:
tfidf = TfidfVectorizer(ngram_range=(1, 2), max_df=0.85, min_df=5)

In [308]:
X_tfidf = tfidf.fit_transform(df['convt_message'])

In [309]:
y = df['target']

In [310]:
from sklearn.model_selection import train_test_split

In [311]:
X_train,X_test,y_train,y_test = train_test_split(X_tfidf,y,test_size=0.2,random_state=4)

In [312]:
print(X_train.shape)
print(X_test.shape)

(4457, 2274)
(1115, 2274)


In [313]:
print(y_train.shape)
print(y_test.shape)

(4457,)
(1115,)


In [314]:
from sklearn.ensemble import RandomForestClassifier

In [315]:
rf = RandomForestClassifier()

In [316]:
rf.fit(X_train,y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [317]:
y_pred_rd = rf.predict(X_test)

In [318]:
accuracy_score(y_test,y_pred_rd)

0.9739910313901345

In [319]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_rd))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98       947
           1       0.97      0.85      0.91       168

    accuracy                           0.97      1115
   macro avg       0.97      0.92      0.95      1115
weighted avg       0.97      0.97      0.97      1115



In [321]:
import pickle

In [322]:
file = open('email_spam.pkl', 'wb')

pickle.dump(rf,file)
file.close()

In [ ]:
with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf, f)